In [1]:
import numpy as np
import tensorflow as tf

train_dir = '00_input/train'
im_size = 100
coords_size = 28

In [2]:
def read_csv(filename):
    from numpy import array
    res = {}
    with open(filename) as fhandle:
        next(fhandle)
        for line in fhandle:
            parts = line.rstrip('\n').split(',')
            coords = array([float(x) for x in parts[1:]], dtype='float64')
            res[parts[0]] = coords
    return res

train_gt = read_csv(train_dir+'/gt.csv')

In [3]:
def parse(train_gt, train_img_dir, info=False):
    from skimage.data import imread
    from scipy.ndimage import zoom
    train_X = np.zeros((len(train_gt), im_size, im_size, 1))
    train_Y = np.zeros((len(train_gt), coords_size))
    for i, img_name in enumerate(train_gt):
        img = imread(train_img_dir+'/'+img_name, as_grey=True)
        train_Y[i] = train_gt[img_name]
        for j in range(0, coords_size, 2):
            train_Y[i][j] *= im_size/img.shape[0]
        for j in range(1, coords_size, 2):
            train_Y[i][j] *= im_size/img.shape[1]
        train_Y[i] = (train_Y[i] / 100) - 0.5
        img = zoom(img, [im_size/img.shape[0], im_size/img.shape[1]])
        img = (img / 255) - 0.5
        train_X[i,:,:,0] = img
        del(img)
        if info and (i+1)%100 == 0:
            print('Image: ', i+1, end='\r')
    return train_X, train_Y

X, Y = parse(train_gt, train_dir+'/images', True)

/home/roman/anaconda3/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:600: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


AttributeError: 'numpy.ndarray' object has no attribute 'save'

In [4]:
np.save('X.npy', X)
np.save('Y.npy', Y)

In [5]:
from keras.models import Model
from keras import backend as K
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint
from keras.utils import Sequence

from keras.layers import (Input, concatenate, Conv2D, MaxPooling2D, 
                          UpSampling2D, Convolution2D, ZeroPadding2D, 
                          BatchNormalization, Activation, concatenate, 
                          Flatten, Dense, merge)
from keras.optimizers import rmsprop

def get_model():
    inputs = Input(shape=(im_size, im_size, 1))
    conv = Conv2D(filters=512, kernel_size=(3,3), padding='same')(inputs)
    batchnorm = BatchNormalization()(conv)
    relu = Activation('relu')(batchnorm)
    
    conv = Conv2D(filters=256, kernel_size=(3,3), padding='same')(relu)
    batchnorm = BatchNormalization()(conv)
    relu = Activation('relu')(batchnorm)
    maxpool = MaxPooling2D()(relu)
    
    conv = Conv2D(filters=128, kernel_size=(3,3), padding='same')(maxpool)
    batchnorm = BatchNormalization()(conv)
    relu = Activation('relu')(batchnorm)
    
    conv = Conv2D(filters=64, kernel_size=(3,3), padding='same')(relu)
    batchnorm = BatchNormalization()(conv)
    relu = Activation('relu')(batchnorm)
    maxpool = MaxPooling2D()(relu)
    
    conv = Conv2D(filters=32, kernel_size=(3,3), padding='same')(maxpool)
    batchnorm = BatchNormalization()(conv)
    relu = Activation('relu')(batchnorm)
    
    flatten = Flatten()(relu)
    predictions = Dense(coords_size, activation='tanh')(flatten)
    
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='rmsprop', loss='mean_squared_error')
    return model

#model = get_model()

In [6]:
#model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 32)      320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 100, 32)      128       
_________________________________________________________________
activation_1 (Activation)    (None, 100, 100, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 64)        18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 50, 50, 64)        256       
__________

In [7]:
def train_detector(train_gt, train_img_dir, fast_train=False):
    train_X, train_Y = parse(train_gt, train_img_dir, True)
    model = get_model()
    if fast_train:
        epochs = 1
    else:
        epochs = 100
    try:
        model.fit(train_X, train_Y, epochs=epochs, batch_size=10, validation_split=0.1)
        model.save_weights('facepoints_model.hdf5')
    except:
        model.save_weights('facepoints_model.hdf5')
    return model

In [8]:
train_detector(train_gt, train_dir+'/images', True)

/home/roman/anaconda3/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:600: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


Train on 5400 samples, validate on 600 samples
Epoch 1/1
 210/5400 [>.............................] - ETA: 16:08 - loss: 0.1532

In [ ]:
def detect():